# Performing text analysis on recipe ingredients + cuisines

**Topics:** Text analysis, classifiers

**Datasets**

- **recipes.csv** Once upon a time I scraped a bunch of recipes, not sure where they're from.

## Imports

We're going to be using

* Pandas
* Regular expressions
* Decision Trees
* Random Forests
* Logistic Regression
* Confusion matrices
* CountVectorizer
* TfidfVectorizer

Import everything you need for those. It's easier than doing them one-by-one later, I think! We're also going to be doing the graphviz tree visualization, but I always just cut and paste that entire block as a unit.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import re
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

In [2]:
%matplotlib inline

## Read in our data

There's nothing too complicated about it.

In [3]:
df = pd.read_csv('recipes.csv')
df

,cuisine,id,ingredient_list
0,greek,10259,"romaine lettuce, black olives, grape tomatoes,..."
1,southern_us,25693,"plain flour, ground pepper, salt, tomatoes, gr..."
2,filipino,20130,"eggs, pepper, salt, mayonaise, cooking oil, gr..."
3,indian,22213,"water, vegetable oil, wheat, salt"
4,indian,13162,"black pepper, shallots, cornflour, cayenne pep..."
5,jamaican,6602,"plain flour, sugar, butter, eggs, fresh ginger..."
6,spanish,42779,"olive oil, salt, medium shrimp, pepper, garlic..."
7,italian,3735,"sugar, pistachio nuts, white almond bark, flou..."
8,mexican,16903,"olive oil, purple onion, fresh pineapple, pork..."
9,italian,12734,"chopped tomatoes, fresh basil, garlic, extra-v..."


# Vectorizing the ingredient list

These words aren't going to do us much good unless we can convert them to text. Use the `CountVectorizer` to count the number of times each word appears in each sentence.

In [4]:
count_vectorizer = CountVectorizer()

In [5]:
X = count_vectorizer.fit_transform(df['ingredient_list'])

In [6]:
X

<39774x3010 sparse matrix of type '<class 'numpy.int64'>'
	with 761951 stored elements in Compressed Sparse Row format>

In [7]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
words_df = pd.DataFrame(X.toarray())
words_df

,0,1,2,3,4,5,6,7,8,9,...,3000,3001,3002,3003,3004,3005,3006,3007,3008,3009
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
words_df.count

<bound method DataFrame.count of        0     1     2     3     4     5     6     7     8     9     ...  3000  \
0         0     0     0     0     0     0     0     0     0     0  ...     0   
1         0     0     0     0     0     0     0     0     0     0  ...     0   
2         0     0     0     0     0     0     0     0     0     0  ...     0   
3         0     0     0     0     0     0     0     0     0     0  ...     0   
4         0     0     0     0     0     0     0     0     0     0  ...     0   
5         0     0     0     0     0     0     0     0     0     0  ...     0   
6         0     0     0     0     0     0     0     0     0     0  ...     0   
7         0     0     0     0     0     0     0     0     0     0  ...     0   
8         0     0     0     0     0     0     0     0     0     0  ...     0   
9         0     0     0     0     0     0     0     0     0     0  ...     0   
10        0     0     0     0     0     0     0     0     0     0  ...     0   
11     

In [10]:
count_vectorizer.get_feature_names()

['00',
 '10',
 '100',
 '14',
 '15',
 '25',
 '33',
 '40',
 '43',
 '95',
 '96',
 'abalone',
 'abbamele',
 'absinthe',
 'abura',
 'acai',
 'accent',
 'accompaniment',
 'achiote',
 'acid',
 'acini',
 'ackee',
 'acorn',
 'acting',
 'activ',
 'active',
 'added',
 'adobo',
 'adzuki',
 'agar',
 'agave',
 'age',
 'aged',
 'ahi',
 'aioli',
 'ajinomoto',
 'ajwain',
 'aka',
 'alaskan',
 'albacore',
 'alcohol',
 'ale',
 'aleppo',
 'alexia',
 'alfalfa',
 'alfredo',
 'all',
 'allspice',
 'almond',
 'almondmilk',
 'almonds',
 'aloe',
 'alphabet',
 'alum',
 'amaranth',
 'amarena',
 'amaretti',
 'amaretto',
 'amba',
 'amber',
 'amberjack',
 'amchur',
 'america',
 'american',
 'aminos',
 'ammonium',
 'amontillado',
 'ampalaya',
 'an',
 'anaheim',
 'anasazi',
 'ancho',
 'anchovies',
 'anchovy',
 'and',
 'andouille',
 'anejo',
 'angel',
 'anglaise',
 'angled',
 'angostura',
 'angus',
 'anise',
 'anisette',
 'anjou',
 'annatto',
 'any',
 'aonori',
 'apple',
 'apples',
 'applesauce',
 'applewood',
 'apricot'

In [11]:
words_df = pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())
words_df

,00,10,100,14,15,25,33,40,43,95,...,za,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Your result should be a dataframe named `words_df` that looks like a much wider version of this:
    
|00|10|...|zucchini|épices|
|---|---|---|---|---|
|0|0|...|0|0|
|0|0|...|0|0|
|0|0|...|0|0|
|0|0|...|0|0|
|0|0|...|0|0|

## Counting

### Count up how many times each word is used

Don't think too hard about this, there's nothing fancy about how you do this.

* **Tip:** Just imagine it's a normal dataframe and you want to add up each column.
* **Tip:** ...or imagine you're trying to add up _one_ column but you forget the column name.

In [12]:
# Not sure which of the two options is the one we want.

In [13]:
words_df.sum()

00                  1
10                  5
100                 1
14                  5
15                  3
25                  3
33                  6
40                 12
43                  1
95                  4
96                  1
abalone             2
abbamele            1
absinthe            1
abura               4
acai                5
accent              8
accompaniment       1
achiote            23
acid                6
acini               3
ackee               9
acorn              18
acting             21
activ               1
active            379
added              49
adobo             314
adzuki              5
agar                8
                 ... 
yams               54
yardlong           22
yeast             758
yellow           2299
yellowfin           4
yellowtail          2
yoghurt           279
yogurt           1167
yolk               14
yolks            1117
yoplait             3
york               20
young               4
yu                  3
yuca      

In [14]:
words_df.sum(axis = 1, skipna = True)

0        16
1        19
2        20
3         5
4        33
5        20
6        24
7        18
8        25
9        14
10       22
11       30
12       27
13       33
14       30
15       16
16        7
17        9
18        8
19       28
20       24
21       11
22       11
23       25
24       17
25       27
26       15
27       21
28       28
29        7
         ..
39744    15
39745    10
39746    33
39747    10
39748    18
39749    26
39750    30
39751    30
39752    11
39753    27
39754    28
39755    21
39756    16
39757    12
39758    31
39759    18
39760    10
39761    34
39762    20
39763    17
39764    29
39765     8
39766    14
39767    29
39768    23
39769    26
39770    18
39771    17
39772    45
39773    23
Length: 39774, dtype: int64

### What are the top 10 most common ingredients?

You'll want to use the results from the last question. Again, don't overthink it.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=False, norm='l1')
X = tfidf_vectorizer.fit_transform(words_df)
pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

,00,10,100,14,15,25,33,40,43,95,...,za,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Does anything seem wrong with those ingredients?

Do they all seem very ingredient-y to you?

In [17]:
# NO! 

### Look at your original recipes dataframe again

What do the ingredients look like? How would you as a _person_ separate them compared to how the vectorizer did it?

In [18]:
df

,cuisine,id,ingredient_list
0,greek,10259,"romaine lettuce, black olives, grape tomatoes,..."
1,southern_us,25693,"plain flour, ground pepper, salt, tomatoes, gr..."
2,filipino,20130,"eggs, pepper, salt, mayonaise, cooking oil, gr..."
3,indian,22213,"water, vegetable oil, wheat, salt"
4,indian,13162,"black pepper, shallots, cornflour, cayenne pep..."
5,jamaican,6602,"plain flour, sugar, butter, eggs, fresh ginger..."
6,spanish,42779,"olive oil, salt, medium shrimp, pepper, garlic..."
7,italian,3735,"sugar, pistachio nuts, white almond bark, flou..."
8,mexican,16903,"olive oil, purple onion, fresh pineapple, pork..."
9,italian,12734,"chopped tomatoes, fresh basil, garlic, extra-v..."


# Using a custom tokenizer

Usually we can trust what the vectorizer does to the words, but this time we can't! Since we have a better idea about how to split it, we use a custom function to split each piece of text up.

To do this: copy and paste your code where you counted words up above. Add `tokenizer=split_on_commas` to `CountVectorizer` to tell it to use my custom function below.

In [19]:
cv = CountVectorizer()
cv_fit=cv.fit_transform(words_df)

cv.get_feature_names()
cv_fit.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [20]:
def split_on_commas(text):
    # Split the list on commas, then remove any extra whitespace
    pieces = text.split(",")
    no_spaces = [piece.strip() for piece in pieces if piece]
    return no_spaces

tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=False, norm='l1')
X = tfidf_vectorizer.fit_transform(words_df)
pd.DataFrame(X.toarray(), tokenizer=split_on_commas, columns=tfidf_vectorizer.get_feature_names())

TypeError: __init__() got an unexpected keyword argument 'tokenizer'

Doesn't that look nicer??

_Yes!_

## Word counting, again

Get the top 15 most common ingredients. **It should be a much nicer list!**

# Using our cuisines

## What are the most popular cuisines?

* **Tip:** You'll need to go back to your original dataframe for this.

In [ ]:
df

In [ ]:
df['cuisine'].value_counts()

## Find every recipe that uses soy sauce

In [ ]:
df[df['ingredient_list'].str.contains('soy sauce', na = False)]

In [ ]:
# Just checking how many they are

In [ ]:
df[df['ingredient_list'].str.contains('soy sauce', na = False)].count()

In [ ]:
df['ingredient_list'].str.contains('soy sauce').value_counts()

## What's are the top 5 cuisines that uses soy sauce?

In [ ]:
soy_sauce = df[df['ingredient_list'].str.contains('soy sauce', na = False)]
soy_sauce

In [ ]:
soy_sauce['cuisine'].value_counts()

Hm, soy sauce seems pretty common in Chinese food. **Maybe we could do something with that?**

# Building a Chinese food detector

Create a new column in your original dataframe called `is_chinese`. It should be `1` if the recipe is Chinese, `0` if the recipe is not.

In [23]:
df['is_chinese'] = df.cuisine.str.contains('chinese', na=False).astype(int)
df['is_chinese']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       1
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       1
24       0
25       1
26       0
27       0
28       1
29       0
        ..
39744    0
39745    0
39746    0
39747    0
39748    0
39749    0
39750    0
39751    0
39752    0
39753    0
39754    0
39755    0
39756    0
39757    0
39758    0
39759    0
39760    0
39761    1
39762    0
39763    0
39764    0
39765    0
39766    0
39767    0
39768    0
39769    0
39770    0
39771    0
39772    1
39773    0
Name: is_chinese, Length: 39774, dtype: int64

In [24]:
df.is_chinese.value_counts()

0    37101
1     2673
Name: is_chinese, dtype: int64

Confirm that you have 2,673 Chinese recipes and 37,101 non-Chinese recipes.

## Building your features

To use a classifier, we need to build an `X` and a `y`.

* Your `X` is going to be the word counts for each recipe
* Your `y` is going to be whether each recipe Chinese or not

This is going to be a little different than what we normally do. Typically we use `.drop(column=` to remove our label, but this time we have **two separate dataframes** we're managing, so don't be surprised if you don't sue it.

In [25]:
x = words_df
y = df.is_chinese

Confirm that `X` is a dataframe of numbers (mostly zeroes) and `y` is a single column of `0`s and `1`s.

In [29]:
x.columns

Index(['00', '10', '100', '14', '15', '25', '33', '40', '43', '95',
       ...
       'za', 'zatarain', 'zatarains', 'zero', 'zest', 'zesty', 'zinfandel',
       'ziti', 'zucchini', 'épices'],
      dtype='object', length=3010)

In [28]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Columns: 3010 entries, 00 to épices
dtypes: int64(3010)
memory usage: 913.4 MB


In [27]:
y.value_counts()

0    37101
1     2673
Name: is_chinese, dtype: int64

### Split into test and train datasets

If we train our classifier on all of our recipes, it might cheat and memorize the answers (aka overfitting). Instead, we'll need to use `train_test_split` to separate into a training set it can study, and a test set we can use to see how well it performs. 

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

ValueError: Found input variables with inconsistent numbers of samples: [3010, 39774]

# Using our classifiers

## Decision Tree

**Create a decision tree classifier, and train it on our training data.** You can make a decision tree classifier like this:

```python
clf = DecisionTreeClassifier()
```

You'll want to also give it `max_depth=5` so we can visualize it without too much trouble later - otherwise it'll make many many decisions which will make a large complicated graphic.

Additionally, put `%%time` in the first row of the cell to see how long it takes to train our classifier.

In [ ]:
clf = DecisionTreeClassifier()
%%time

### Print out our confusion matrix 

Use the one from our notes and homework, not the default scikit-learn version. The output should look something like this:

|.|Predicted not chinese|Predicted chinese|
|---|---|---|
|Is not chinese|9209|60|
|Is chinese|377|298|

**How do we feel about our classifier's performance?** Describe what is happening in the "is chinese" row using words.

### What are the important features?

You can use the following code to see feature importance for a decision tree.

```python
feature_names = X.columns
importances = clf.feature_importances_

pd.DataFrame({
    'feature': feature_names,
    'feature importance': importances,
}).sort_values(by='feature importance', ascending=False)
```

We gave it a lot of columns to pay attention to, so **why are so many of them zero?**

### Visualize the decision tree

While those numbers are fun (if a little long), the _real fun_ is visualizing the tree itself. You can use this code to do that:

```python
from sklearn import tree
import graphviz

label_names = ['not chinese', 'chinese']
feature_names = X.columns

dot_data = tree.export_graphviz(clf,
                    feature_names=feature_names,  
                    filled=True,
                    class_names=label_names)  
graph = graphviz.Source(dot_data)  
graph
```

**What do you think about that?** If you know about cooking or ingredients, do you recognize what some of those splits mean in terms of different cuisines?

## Random Forest

A random forest is like making a bunch of decision trees work together. **Create a random forest classifier, and train it on our data.**

```python
clf = RandomForestClassifier()
```

Right now, the `RandomForestClassifier` uses 10 decision trees to make its decision, with `n_estimators=10` being the default. You'll see a warning about how it's going to change to `100` in a future version.

You can try to use `100` now, but notice how long it takes! Use `%%time` to time how long it takes to train the random forest.

### What are the feature importances?

You can re-use your decision tree code. Are the results any different?

### Why can't we draw a tree for a random forest?

# Classifying with a logistic regression

To be a real data scientist, you throw every kind of classifier at your project until you find the one that works the best. Is that science? I don't know, really, but it's definitely reality.

## Building our classifier

This is the standard way to build a logistic regression classifier:

```python
clf = LogisticRegression(C=1e9, solver='lbfgs')
```

> You might get an error when fitting, though, that says "Increase the number of iterations." This means the regression failed to find a good answer, and you need to tell it to work harder. To do that, you need to set `max_iter=` to a higher number.
>
> The default is 100, and in previous homeworks we've cranked it up to 4000 with no problem. This time it isn't so easy - we have a lot lot lot lot of features this time, though, and every iteration makes the regression take more time until we get very very bored of things.
>
> You can solve this three ways:
>
> * Scroll up and add a `max_features=` to your vectorizer so it only picks a certain (lower) number of features, then re-run
> * Be very patient
> * Train on less data, by doing for example `clf.fit(X_train.head(7000), y_train.head(7000))`
> 
> I used `%%time` to see how long .fit would take with `max_iter=1000` - it ran for 5 minutes before it finished, but it never converged (aka got an answer)! My computer's old, but if you try that route you should go get a snack.

**Build a logistic classifier and train it on our training data**

### Examining the coefficients

Use this code to print out the coefficients, it's slightly different than the decision tree version:

```python
import numpy as np

feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)
```

I might throw a `.head(20)` on there so you only see the top ones.

These are pretty different from our decision tree.

**Why might these numbers be so different compared to our decision tree?** Think about how decision trees and logistic regression work. Or maybe go answer the next question and then come back.

**Use words to describe what the odds ratio for your top ingredient means.**

### Check the confusion matrix

How well does it do compared to the previous two classifiers?

Describe in words how well it does at accurately predicting Chinese food recipes as being Chinese.

### Which was your best classifier so far?

Do you have to think about what "best" means, or is these a clear winner?

# Upgrading to a TF-IDF Vectorizer

Vectorizing using TF-IDF is almost always better than using simple counts. You can't find the most popular words or anything, but common words get downgraded and less common, more descriptive words end up rising in importance. And that seems cool!

Take your `CountVectorizer` code from up above and change it to use a `TfidfVectorizer`. I'll leave the `split_on_commas` code here as a start.

In [ ]:
def split_on_commas(text):
    pieces = text.split(",")
    no_spaces = [piece.strip() for piece in pieces if piece]
    return no_spaces


## Building our features

Create our `X` and `y` variables. Our `X` should be the adjusted word frequencies, and `y` should be whether the recipe is Chinese or not.

Once that's done, be sure to split your features and labels into test and train datasets.

## Decision trees

**Create a decision tree and train it on our data.** Time how long it takes to complete training.

## What does it use to make its decisions?

You can either show me feature importances or the actual tree that's built (if you're using a low enough `max_features`).

Compare this with what you got with the CountVectorizer. Do they seem that different?

### How well does this classifier do?

Confusion matrix, please.

Does it perform better than the `CountVectorizer` version?

# Logistic Regression classifier

I don't think we did that much better - if any better - with our new version. Let's try one more time using logistic regression.

If you get the "could not converge, increase iterations" error, refer to the tips up above about `max_iter` and `max_features`.

# Questions

### Question 1: What is "vectorization?" Why do we do it?

### Question 2: What's the general difference between a `CountVectorizer` and a `TfidfVectorizer`?

### Question 3: Why did we end up using a custom tokenizer instead of the default scikit-learn one?

### Question 4: What's the best combination of vectorizer + classifier we've used? Does "best" depend on what's important to us?

### Question 5: How do you feel about trying out random classifiers until you find the "best" one?

### Question 6: What's the most "Chinese" ingredient? You might have a different answer based on what type of classifier you're using - why is that?


### Question 7: If you look at the top few words for the random forest, most of them are various types of soy sauce. How could you improve upon that when if you were spending more time with this dataset?

### Question 8: Scikit-learn vectorizers have a lot of options. Among those are `max_features`, which only takes the top n features, and `min_df`/`max_df`, which only allows words that show up above or below a certain percentage of the documents. For example, `max_df` of 0.75 removes any words that show up in more than 75% of documents. When might these options be useful?


### Question 9: If you had to explain a classifier to someone, would you rather explain a logistic regression or a decision tree? Why is that?

### Question 10: A random forest performs better than a decision tree, but it's easy to see how a decision tree works by looking at the flow chart. If you had to explain to your readers the algorithm behind your analysis, would you choose a better-performing algorithm or one that is easier to explain?

# Bonus!!!

## Multi-class problems

Did you know that `DecisionTreeClassifier` can actually support multiple classes, not just two? Probably not, since I've kept it a secret. If you're interested, let's try this out.

### Convert categories to numbers

First, convert the `cuisine` column into a category with `.astype('category')`

In [ ]:
df['cuisine'].astype('category')

The cuisine looks like a string, but it's now secretly a number. You can use `.cat.codes` to get the code out. Assign that to a new column called `cuisine_code`

In [ ]:
df['type_code'] = df.type.cat.codes
df[['type', 'type_code']].head(10)

**Create a new `X` and `y`**. The `X` is the vectorized words, and the `y` is the cuisine code.

**Test train split!**

**Build a decision tree** and train it on the training data.

The confusion matrix code is a little different, but here you go:

```python
y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(df.cuisine.cat.categories)
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)
```